In [72]:

import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl import workbook
import xlrd
import os
import numpy as np
import datetime as dt

# file pathes to b2w report "Cost Detail With Job Cost ID" and to where it should get saved once modified 
new_file_Path = r"C:\Users\will.davis\Desktop\Estimate reports\Cost Detail With Job Cost ID1.xlsx"
og_path = r"C:\Users\will.davis\Desktop\Estimate reports\Cost Detail With Job Cost ID.xlsx"
# list of lists containg reference file paths, the column to search, and the category that should be assigned 
estimate_reference_paths = [[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_pay_item_db.xlsx"), "name", "Pay Item"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_task_db.xlsx"), "Task Name", "Task"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_crew_db.xlsx"), "Crew Name", "Crew"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_materials_db.xlsx"),"Material Type", "Material"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_misc_db.xlsx"),"Name", "Misc."],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_labor_db.xlsx"), "Labor Type", "Labor"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_equipment_db.xlsx"),"Equipment Name", "Equpment"]]

# Read excel workbook of Cost Detail With Job Cost ID
df1 = pd.read_excel(og_path, skiprows=12)
# Remove blank columns/rename columns/dropna
df2 = df1.drop(df1.columns[[2,6,7,8,9,10,11,13,14,16]], axis=1)
df2.rename(columns={"Unnamed: 1":"Pay Item", "Unnamed: 3":"Task/Componenet", "Unnamed: 4":"Crew/Componenet", "Unnamed: 5":"Labor/Equipment",
                          "Direct Cost":"Unit Direct Cost","Direct Cost.1":"Total Direct Cost"},inplace=True)
df2.dropna(subset="Unit Direct Cost", inplace=True)
df3 = df2[df2["Unit Direct Cost"].str.contains("$")]
# fill blank pay items with item above for hiararchy
df3['Pay Item'] = df3['Pay Item'].ffill()
df3[["Category","Hierarchy #", "Hierarchy Structure"]] = "Custom","",""
# create blank columns for different flags
df3[["UM Cost ID","UM Used", "Material Inside Task","Has Cost ID", "Is LS","Recomended Cost ID"]] = ""
# establish categories and hierarchy 
for index, row1 in df3.iterrows():
    hierarchy = 0 
    for col1 in range(1,5):
        hierarchy += 1
        if pd.notna(row1.iloc[col1]):
            no_bracket_value = str(row1.iloc[col1]).split(" [")[0]
            # print(no_bracket_value)
            for cat in estimate_reference_paths:

# need a method to identify crews as crews 

                # print(row1.iloc[col1])
                # if cat[2] == "Crew":
                #     no_bracket_value = no_bracket_value.split("(")[0]
                if no_bracket_value in list(cat[0][cat[1]]):
                    df3.at[index,"Category"] = cat[2]
                    df3.at[index,"Hierarchy #"] = hierarchy



In [73]:
# flag items with same cost ID but different Ums that are not materials or misc
# flag all, will adjust for subbed and materials thot don't matter
cost_id_list = list(df3["Job Cost ID"].unique())
for cost_id in cost_id_list:
    df_filtered = df3[df3["Job Cost ID"] == cost_id]
    UM_list = list(df_filtered['UM'].unique())
    UM_str = " ,".join(UM_list)
    if len(UM_list) != 1:
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Cost ID"] = "Different"
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Used"] = UM_str
    else:
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Cost ID"] = "Same"
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Used"] = UM_str


In [74]:

# Flag materials inside of tasks or other items 
def material_in_task_col(row):
    if row["Category"] == "Material" and row["Hierarchy #"] > 2:
        return 'Inside Task'
    else:
        return " "
# Flag materials without cost ID
def material_no_cost_ID(row):
    if row["Category"] == "Material" or row["Category"] == "Task" and row["Job Cost ID"] == "":
        return 'No'
    else:
        return " "

# Flag LS items
def is_ls_um(row):
    if row["UM"] == "LS":
        return 'Yes'
    else:
        return " "

df3 = df3.fillna("")
df3["Material Inside Task"] = df3.apply(material_in_task_col, axis=1)
df3["Has Cost ID"] = df3.apply(material_no_cost_ID, axis=1)
df3["Is LS"] = df3.apply(is_ls_um, axis=1)




In [75]:
# df3.head(5)
df3[df3['UM']=="LS"]

,Description,Pay Item,Task/Componenet,Crew/Componenet,Labor/Equipment,Job Cost ID,Quantity,UM,Unit Direct Cost,Total Direct Cost,Category,Hierarchy #,Hierarchy Structure,UM Cost ID,UM Used,Material Inside Task,Has Cost ID,Is LS,Recomended Cost ID
493,,BLDGDEMO - BUILDING DEMOLITION,020.310. Miscellaneous Demo,,,020.310.,1.00,LS,"$3,661.12","$3,661.12",Task,2,,Same,LS,,,Yes,
494,,BLDGDEMO - BUILDING DEMOLITION,,"Medium Demo (10.00 PH/LS, 1.00 DY)",,,1.00,LS,"$3,661.12","$3,661.12",Custom,,,,,,,Yes,
507,,BLDGDEMO - BUILDING DEMOLITION,010.400. Dump Fees,,,010.400.,1.00,LS,"$39,480.00","$39,480.00",Task,2,,Different,"LS ,LOAD",,,Yes,
590,,BONDS - Bonds,,,,,0.00,LS,$0.00,$0.00,Custom,,,,,,,Yes,
591,,BONDS - Bonds,010.110. Bonds,,,010.110.,0.00,LS,$0.00,$0.00,Task,2,,Same,LS,,,Yes,
1298,,DEMOSAN - Cut Sewer Main In Street,065.020. Asphalt Sawing - Sub,,,065.020.,1.00,LS,$325.00,$325.00,Task,2,,Different,"LF ,LS",,,Yes,
1301,,DEMOSAN - Cut Sewer Main In Street,065.165. Private Locator - Sub,,,065.165.,1.00,LS,$600.00,$600.00,Task,2,,Same,LS,,,Yes,
1420,,ECGC - EC General Conditions,,,,,0.00,LS,$0.00,$0.00,Custom,,,,,,,Yes,
1434,,ECGC - EC General Conditions,,Procore (.12% Of Total Estimate),,,0.00,LS,$0.00,$0.00,Custom,,,,,,,Yes,
1466,,ECMOB - EC Mobilization,,,,,0.00,LS,$0.00,$0.00,Custom,,,,,,,Yes,


In [69]:
# df3[df3['Hierarchy #'] == 1]
df3[df3['Category'] == "Task"].head(5)

,Description,Pay Item,Task/Componenet,Crew/Componenet,Labor/Equipment,Job Cost ID,Quantity,UM,Unit Direct Cost,Total Direct Cost,Category,Hierarchy #,Hierarchy Structure,UM Cost ID,UM Used,Material Inside Task,Missing Cost ID,Recomended Cost ID,Has Cost ID
1,,AD - Area Drain / Inlet,080.370. Area Inlet,,,080.370.,0.00,EACH,$0.00,$0.00,Task,2,,Same,EACH,,,,
13,,AD - Area Drain / Inlet,050.343. Truck Storm Bedding Material **NEW**,,,050.343.,0.00,TON,$0.00,$0.00,Task,2,,Same,TON,,,,
16,,AD - Area Drain / Inlet,080.491. Storm Bedding Material ONLY,,,,0.00,TON,$0.00,$0.00,Task,2,,,,,,,No
20,,BEND1045 - 10 Inch Watermain Bend 45 Deg.,"080.555. 10"" Water (DIP/C900)",,,080.555.,0.00,LF,$0.00,$0.00,Task,2,,Same,LF,,,,
39,,BEND04 - 04 INCH Watermain Bends,"080.540. 04"" Water (DIP/C900)",,,080.540.,0.00,LF,$0.00,$0.00,Task,2,,Same,LF,,,,


In [ ]:
#to do
# flag potentially wrong Cost ID when compared to Cost Id reference sheet from Jill 
# flag potentially custom Items, they may need complete restructure
# flag pay items that have taskes with different work types

# checking quipment and labor rates, flag old rates if there is a delta between current 
# flag no cost code on little blue truck
# flag cost ID(first 3 digits) that has the greatest value in its pay item 

# some what complete
# flag items with same cost ID but different Ums that are not materials or misc  ## DONE
# Flag materials inside tasks # i think this is correct 
# flag materials that do not have Cost IDs
# flag all lump sum items 

# not pressing 
# flag items that have UM of Day/Month/LS that a foreman can charge to back burner 

